## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import tensorflow as tf

%run helper_functions.ipynb

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

__Target Variable__: `IS_SUCCESSFUL`

__Feature Variables__: All other columns

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Limit columns to 10 unique values
limit_unique(
    application_df,
    max_value = 10,
    columns_to_limit = ['APPLICATION_TYPE', 'CLASSIFICATION']
)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64
Number of unique values: 10

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      887
C7000      777
C1700      287
C4000      194
C5000      116
Name: count, dtype: int64
Number of unique values: 10



### Map to Binary Integers

In [5]:
# Since binary, convert SPECIAL_CONSIDERATIONS to 0s and 1s.
print(f"Before conversion: {application_df['SPECIAL_CONSIDERATIONS'].value_counts()}\n")

application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].map({
    'N': 0,
    'Y': 1
})
print(f"After conversion: {application_df['SPECIAL_CONSIDERATIONS'].value_counts()}")

Before conversion: SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

After conversion: SPECIAL_CONSIDERATIONS
0    34272
1       27
Name: count, dtype: int64


### Ordinal Encoding
- There is no data on: `500000-1M`
- Solution: Convert to ordinal encoding to account for this

In [6]:
# Convert each value to an ordinal score
income_map = {
    '0': 0,
    '1-9999': 1,
    '10000-24999': 2,
    '25000-99999': 3,
    '100000-499999': 4,
    '500000-1M': 5, # No data in this category
    '1M-5M': 6,
    '5M-10M': 7,
    '10M-50M': 8,
    '50M+': 9
}
application_df['ORDINAL_INCOME_AMT'] = application_df['INCOME_AMT'].map(income_map)

In [7]:
# Drop the INCOME_AMT column
application_df = application_df.drop(columns=['INCOME_AMT'])

### Encoding

In [8]:
# Get all the columns which are categorical
categorical_hot = list(application_df.select_dtypes(include="object").columns)

# Convert categorical data to numeric with `pd.get_dummies`
encoded_columns = pd.get_dummies(application_df[categorical_hot]).astype(int)
encoded_columns.head()

APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  \
0                       0                     1                     0   
1                       0                     0                     0   
2                       0                     0                     0   
3                       0                     0                     0   
4                       0                     0                     0   

   APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  \
0                    0                    0                    0   
1                    1                    0                    0   
2                    0                    0                    1   
3                    1                    0                    0   
4                    1                    0                    0   

   APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  APPLICATION_TYPE_T8  \
0                    0                    0                    0   
1                    0                    0                    0   
2                    0                    0                    0   
3                    0                    0                    0   
4                    0                    0                    0   

   APPLICATION_TYPE_T9  ...  CLASSIFICATION_Other  USE_CASE_CommunityServ  \
0                    0  ...                     0                       0   
1                    0  ...                     0                       0   
2                    0  ...                     0                       0   
3                    0  ...                     0                       0   
4                    0  ...                     0                       0   

   USE_CASE_Heathcare  USE_CASE_Other  USE_CASE_Preservation  \
0                   0               0                      0   
1                   0               0                      1   
2                   0               0                      0   
3                   0               0                      1   
4                   1               0                      0   

   USE_CASE_ProductDev  ORGANIZATION_Association  ORGANIZATION_Co-operative  \
0                    1                         1                          0   
1                    0                         0                          1   
2                    1                         1                          0   
3                    0                         0                          0   
4                    0                         0                          0   

   ORGANIZATION_Corporation  ORGANIZATION_Trust  
0                         0                   0  
1                         0                   0  
2                         0                   0  
3                         0                   1  
4                         0                   1  

[5 rows x 35 columns]

In [9]:
# Split our preprocessed data into our features and target arrays
# Isolate the target array
y = application_df['IS_SUCCESSFUL']

# Isolate the feature array
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Replace features with encoded equivalents
X.drop(categorical_hot, axis=1, inplace=True)

# Calculate updated feature array
X = pd.concat([X, encoded_columns], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer0 = 80
hidden_nodes_layer1 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer0,
    input_dim = number_input_features,
    activation = "relu"
))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer1,
    activation = "relu"
))

# Output layer
nn.add(tf.keras.layers.Dense(
    units = 1,
    activation = "sigmoid"
))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3200      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5661 (22.11 KB)
Trainable params: 5661 (22.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [13]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs = 100,
    verbose = 1
)

Epoch 1/100
804/804 [==============================] - 1s 531us/step - loss: 0.5693 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 0s 507us/step - loss: 0.5557 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 0s 518us/step - loss: 0.5537 - accuracy: 0.7285
Epoch 4/100
804/804 [==============================] - 0s 504us/step - loss: 0.5526 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 0s 505us/step - loss: 0.5511 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 0s 507us/step - loss: 0.5504 - accuracy: 0.7300
Epoch 7/100
804/804 [==============================] - 0s 507us/step - loss: 0.5498 - accuracy: 0.7306
Epoch 8/100
804/804 [==============================] - 0s 502us/step - loss: 0.5498 - accuracy: 0.7288
Epoch 9/100
804/804 [==============================] - 0s 505us/step - loss: 0.5481 - accuracy: 0.7317
Epoch 10/100
804/804 [==============================] - 0s 503us/step - l

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5502 - accuracy: 0.7370 - 134ms/epoch - 500us/step
Loss: 0.550153374671936, Accuracy: 0.7370262145996094


In [15]:
# Export our model to HDF5 file
nn.save('models/AlphabetSoupCharity_iteration2b.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
